In [45]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import dask

In [46]:
#datapath = "/mnt/toparctic-ns9869k-ns9252k-blomchannel/BLOM_channel_new31/"
datapath = "/mnt/toparctic-ns9869k-ns9252k-blomchannel/BLOM_channel_new05_mix1/"

In [47]:
pwd

'/mnt/toparctic-ns9869k/alsjur'

In [48]:
dx = 2e3             # [m]
dy = 2e3             # [m]
rho = 1e3            # stemmer dette?
dt = 1*24*60*60      # [s]
f0 = 1e-4            # [s-1]

In [49]:
xchunk = -1
ychunk = -1
sigmachunk = -1
timechunk = 30

In [50]:
# read data for one month, for testing on small data set
ds = xr.open_mfdataset(datapath+"*hd_2027.01.nc", 
                       chunks={"x":xchunk, "y":ychunk, "sigma":sigmachunk, "time":timechunk}
                      )
pp = xr.Dataset()

In [51]:
def xface2center(da):
    ni = len(da.x)

    dac = xr.concat([da.isel(x=slice(0,ni-1)), da.isel(x=slice(1,ni))], dim="temp").mean(dim="temp")
    dacend = xr.concat([da.isel(x=0), da.isel(x=-1)], dim="temp").mean(dim="temp")
    print(dac)
    print(dacend)
    dac = xr.concat([dac, dacend], dim="x")

    return dac

def yface2center(da):
    nj = len(da.y)

    dac = xr.concat([da.isel(y=slice(0,nj-1)), da.isel(y=slice(1,nj))], dim="temp").mean(dim="temp")
    dacend = da.isel(y=-1)*np.nan
    dac = xr.concat([dac, dacend], dim="y")

    return dac

def center2xface(da):
    ni = len(da.x)

    dax = xr.concat([da.isel(x=slice(0,ni-1)), da.isel(x=slice(1,ni))], dim="temp").mean(dim="temp")
    daxfirst = xr.concat([da.isel(x=0), da.isel(x=-1)], dim="temp").mean(dim="temp")
    print(dax)
    print(daxfirst)
    dax = xr.concat([daxfirst, dax], dim="x")

    return dax

def center2yface(da):
    nj = len(da.y)

    day = xr.concat([da.isel(y=slice(0,nj-1)), da.isel(y=slice(1,nj))], dim="temp").mean(dim="temp")
    dayfirst = da.isel(y=0)*np.nan
    day = xr.concat([dayfirst, day], dim="y")

    return day

In [52]:
uflx = ds.uflx
vflx = ds.vflx

dzx = center2xface(ds.dz)
dzy = center2yface(ds.dz)

Ax = dy*dzx
Ay = dx*dzy

u = uflx/(rho*Ax)
v = vflx/(rho*Ay)

<xarray.DataArray 'dz' (time: 30, sigma: 53, y: 512, x: 207)>
dask.array<mean_agg-aggregate, shape=(30, 53, 512, 207), dtype=float32, chunksize=(30, 53, 512, 207), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2027-02-01 12:00:00 ... 2027-02-30 12:00:00
  * sigma    (sigma) float64 33.21 33.21 33.54 33.84 ... 35.79 35.79 35.79 35.8
Dimensions without coordinates: y, x
<xarray.DataArray 'dz' (time: 30, sigma: 53, y: 512)>
dask.array<mean_agg-aggregate, shape=(30, 53, 512), dtype=float32, chunksize=(30, 53, 512), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2027-02-01 12:00:00 ... 2027-02-30 12:00:00
  * sigma    (sigma) float64 33.21 33.21 33.54 33.84 ... 35.79 35.79 35.79 35.8
Dimensions without coordinates: y


In [53]:
u = ds.uvel
v = ds.vvel

uc = xface2center(u)
vc = yface2center(v)

ds["uc"] = uc
ds["vc"] = vc

<xarray.DataArray 'uvel' (time: 30, sigma: 53, y: 512, x: 207)>
dask.array<mean_agg-aggregate, shape=(30, 53, 512, 207), dtype=float32, chunksize=(30, 53, 512, 207), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2027-02-01 12:00:00 ... 2027-02-30 12:00:00
  * sigma    (sigma) float64 33.21 33.21 33.54 33.84 ... 35.79 35.79 35.79 35.8
Dimensions without coordinates: y, x
<xarray.DataArray 'uvel' (time: 30, sigma: 53, y: 512)>
dask.array<mean_agg-aggregate, shape=(30, 53, 512), dtype=float32, chunksize=(30, 53, 512), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2027-02-01 12:00:00 ... 2027-02-30 12:00:00
  * sigma    (sigma) float64 33.21 33.21 33.54 33.84 ... 35.79 35.79 35.79 35.8
Dimensions without coordinates: y


In [54]:
xface2center(ds.uvel)
dzx = center2xface(ds.dz)

<xarray.DataArray 'uvel' (time: 30, sigma: 53, y: 512, x: 207)>
dask.array<mean_agg-aggregate, shape=(30, 53, 512, 207), dtype=float32, chunksize=(30, 53, 512, 207), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2027-02-01 12:00:00 ... 2027-02-30 12:00:00
  * sigma    (sigma) float64 33.21 33.21 33.54 33.84 ... 35.79 35.79 35.79 35.8
Dimensions without coordinates: y, x
<xarray.DataArray 'uvel' (time: 30, sigma: 53, y: 512)>
dask.array<mean_agg-aggregate, shape=(30, 53, 512), dtype=float32, chunksize=(30, 53, 512), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2027-02-01 12:00:00 ... 2027-02-30 12:00:00
  * sigma    (sigma) float64 33.21 33.21 33.54 33.84 ... 35.79 35.79 35.79 35.8
Dimensions without coordinates: y
<xarray.DataArray 'dz' (time: 30, sigma: 53, y: 512, x: 207)>
dask.array<mean_agg-aggregate, shape=(30, 53, 512, 207), dtype=float32, chunksize=(30, 53, 512, 207), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2027-02-

In [56]:
#ds["x"] = dx*np.arange(0,len(ds.x))
#ds["y"] = dy*np.arange(0,len(ds.y))
ds.attrs = {"test":"hei"}
ds.ubaro

<xarray.DataArray 'ubaro' (time: 30, y: 512, x: 208)>
dask.array<open_dataset-ubaro, shape=(30, 512, 208), dtype=float32, chunksize=(30, 512, 208), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2027-02-01 12:00:00 ... 2027-02-30 12:00:00
Dimensions without coordinates: y, x
Attributes:
    units:          m s-1
    long_name:      Barotropic velocity x-component
    coordinates:    ulon ulat
    cell_measures:  area: uarea

In [9]:
# calculate momentum advection
ds["uv"] = ds.uc*ds.vc

In [10]:
# calculate depth integral
#dz = (ds.depth_bnds.isel(bounds=1)-ds.depth_bnds.isel(bounds=0))
dz = ds.dz
pp["UV"] = (ds.uv*dz).sum("sigma")

pp["U"] = (ds.uvel*dz).sum("sigma")
pp["fV"] = f0*(ds.vvel*dz).sum("sigma")

In [11]:
pp["ubar"] = ds.ubaro

In [12]:
# calculate depth H. Total water height - sea surface elevation
H = (ds.dz.sum(dim = 'sigma') - ds.sealv).isel(time=0)#.mean("time")            # [m]

# pad H in reentranse direction
Hpad = xr.concat([H.isel(x=-1), H, H.isel(x=0)], dim="x").chunk({"x":-1, "y":-1})
Hpad["x"] = dx*np.arange(-1,len(ds.x)+1)

# Calculate derivative of bottom height using central difference    
dhdx = -Hpad.differentiate("x").isel(x=slice(1,-1))#/dx
#Hpad_perturb = Hpad - np.mean(Hpad, axis=1)[:,None]



#dHdx = (Hpad[:,2:]-Hpad[:,:-2])/(2*dx)

# Calculate derivative of bottom height using froward difference 
#dHdx = (Hpad[:,2:]-Hpad[:,1:-1])/dx

ds["dhdx"] = dhdx

In [13]:
# calculate elements of topographic form stress
#pbot_perturb = ds.pbot - ds.pbot.mean(dim = 'x')
#pp["phidHdx"] = pbot_perturb*ds.dHdx/rho

pp["phidhdx"] = ds.pbot*ds.dhdx/rho

In [14]:
# calculate momentum flux divergence, second order difference
dUVdy = pp.UV.differentiate("y")#/dy

pp["dUVdy"] =  dUVdy

In [15]:
dz

<xarray.DataArray 'dz' (time: 360, sigma: 53, y: 512, x: 208)>
dask.array<concatenate, shape=(360, 53, 512, 208), dtype=float32, chunksize=(30, 53, 512, 208), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2027-02-01 12:00:00 ... 2028-01-30 12:00:00
  * sigma    (sigma) float64 33.21 33.21 33.54 33.84 ... 35.79 35.79 35.79 35.8
  * x        (x) float64 0.0 2e+03 4e+03 6e+03 ... 4.1e+05 4.12e+05 4.14e+05
  * y        (y) float64 0.0 2e+03 4e+03 6e+03 ... 1.018e+06 1.02e+06 1.022e+06
Attributes:
    units:          m
    long_name:      Layer thickness
    coordinates:    plon plat
    cell_measures:  area: parea

In [16]:
con = np.where(dz>1)

In [27]:
con[0][-1]

359

In [25]:
con[1]

array([ 0,  0,  0, ..., 45, 45, 45])

In [24]:
con[2]

array([  1,   1,   1, ..., 137, 137, 137])

In [23]:
con[3]

array([ 0,  1,  2, ...,  6,  7, 10])

In [29]:
# calculate bottom drag
cbar = 0.05 # is RMS flow speed for linear bottom friction law in [m s-1].
cb = 0.002  # is Coefficient of quadratic bottom friction [unitless].


# from ubbl.py
# dette forstår jeg ikke helt. Hva gjør np.where i dette tilfellet? Hvorfor gir den en liste med 34 arrays, der vi bare bruker første?
def bottom_vel(u,dz):
    bi=np.where(dz>1)[0]
    if len(bi>0):
        return u[bi[-1]]
    else:
        return np.nan


ub = xr.apply_ufunc(bottom_vel, ds.uc, ds.dz,
                                      input_core_dims=[['sigma'],['sigma']],
                                      output_core_dims=[[]],
                                      vectorize=True,
                                      dask='parallelized',
                                      output_dtypes=[ds.uvel.dtype])
    
vb = xr.apply_ufunc(bottom_vel, ds.vc, ds.dz,
                                  input_core_dims=[['sigma'],['sigma']],
                                  output_core_dims=[[]],
                                  vectorize=True,
                                  dask='parallelized',
                                  output_dtypes=[ds.vvel.dtype])
q = cb*(np.sqrt(ub**2+vb**2)+cbar)
tauxb = ub*q

pp["tauxb"] = tauxb

In [ ]:
# zonal mean
results = pp.mean("x")

In [ ]:
# calculate time derivative of velocity, second order difference
dUdt = results.U.differentiate("time", datetime_unit="s")

results["dUdt"] = dUdt

In [ ]:
# calculate bathymetry
bath = -H.mean("x")
results["bath"] = bath

In [ ]:
# calculate time mean for plotting
rm = results.mean("time")
rm

In [ ]:
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

colors = mpl.colormaps['tab10'].resampled(6).colors

fig, ax = plt.subplots()
y = rm.y
surface_forcing = -0.05/rho 
MFD = rm.dUVdy
TFS = rm.phidhdx
dUdt = rm.dUdt
bottom_drag = rm.tauxb
all = MFD+TFS+dUdt+bottom_drag+surface_forcing

ax2 = ax.twinx()
ax2.plot(y, bath, color="lightgray", ls="--")
#ax2.set_ylabel("Depth [m]")
ax2.set_yticks([])

ax.axhline(0, color="gray")
ax.axhline(surface_forcing, label="surface drag", color=colors[0])

vars = [MFD, dUdt, TFS, bottom_drag, all]
labels = ["MFD", "dUdt", "TFS", "bottom drag", "sum"]

for var, label, color in zip(vars, labels, colors[1:]):
    ax.plot(y, var, label=label, color=color)
ax.legend()